In [1]:
import pandas as pd
import numpy as np

In [2]:
DB_PATH = "Simulation/FSN_Data_5k.db"

In [3]:
from NetEmbs.utils.IO.db_connection import upload_data, upload_journal_entries
from NetEmbs.DataProcessing import *

In [4]:
data = upload_data(DB_PATH)

In [5]:
d = data_preprocessing(data)

In [ ]:
data.head(18).to_dict()

In [6]:
from NetEmbs import *
GLOBAL_FSN = FSN()
GLOBAL_FSN.build(d, left_title="FA_Name")

In [ ]:
import random
random.choices(range(len(['NoisyLeftFA_kbfq', 'NoisyLeftFA_phtv', 'NoisyLeftFA_qapc', 'TradePayables'])), 
               weights=[4.05996574e-04, 1.23627551e-03, 1.89143324e-03, 9.96466295e-01], k=1)

In [ ]:
sub_step_one(GLOBAL_FSN, 7, "IN", weighted=True)